# Author Modeling Pipeline

This notebook implements a 5-stage pipeline for modeling an author's characteristic writing patterns through few-shot example curation.

## Methodology

Rather than extracting explicit style rules, this approach models the author's **decision-making patterns** and **sensibility**, then curates exemplary passages for tacit transmission via few-shot learning.

## Pipeline Stages

0. From a set of template texts, a sample of paragraphs are drawn. Cardinality N.
1. Each text sample is analyzed from three perspectives; implied author, decision patterns by author, functional texture. This entails invoking LLMs, cardinality 3N.
2. Each perspective is integrated and synthesized across all texts in order to arrive at some total view of said three perspectives. Cardinality 3.
3. The three total perspectives are combined and transformed in order to model the author's writing mind, thus producing an "incantation" that would make an LLM write like the analyzed author.

## Note

This pipeline is author-agnostic. The Russell corpus is used as a test case, but the same approach applies to any author, public or not.

## Setup & Dependencies

In [18]:
!pip install -r requirements.txt

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [19]:
try:
    import litellm
    print('Providers\n=========')
    print('* ' + '\n* '.join(litellm.LITELLM_CHAT_PROVIDERS))
except ImportError as e:
    print(f"✗ Cannot import litellm: {e}")

Providers
* openai
* openai_like
* bytez
* xai
* custom_openai
* text-completion-openai
* cohere
* cohere_chat
* clarifai
* anthropic
* anthropic_text
* replicate
* huggingface
* together_ai
* datarobot
* openrouter
* cometapi
* vertex_ai
* vertex_ai_beta
* gemini
* ai21
* baseten
* azure
* azure_text
* azure_ai
* sagemaker
* sagemaker_chat
* bedrock
* vllm
* nlp_cloud
* petals
* oobabooga
* ollama
* ollama_chat
* deepinfra
* perplexity
* mistral
* groq
* nvidia_nim
* cerebras
* baseten
* ai21_chat
* volcengine
* codestral
* text-completion-codestral
* deepseek
* sambanova
* maritalk
* cloudflare
* fireworks_ai
* friendliai
* watsonx
* watsonx_text
* triton
* predibase
* databricks
* empower
* github
* custom
* litellm_proxy
* hosted_vllm
* llamafile
* lm_studio
* galadriel
* gradient_ai
* github_copilot
* novita
* meta_llama
* featherless_ai
* nscale
* nebius
* dashscope
* moonshot
* v0
* heroku
* oci
* morph
* lambda_ai
* vercel_ai_gateway
* wandb
* ovhcloud
* lemonade


In [20]:
import os
from pathlib import Path
from pprint import pprint
from typing import List, Dict, Any

from belletrist import (
    LLM,
    LLMConfig,
    PromptMaker,
    DataSampler,
    ResultStore,
    
    # Stage 1 configs
    ImpliedAuthorConfig,
    DecisionPatternConfig,
    FunctionalTextureConfig,
    
    # Stage 2 configs
    ImpliedAuthorSynthesisConfig,
    DecisionPatternSynthesisConfig,
    TexturalSynthesisConfig,
    
    # Stage 3 config
    AuthorModelDefinitionConfig,
)

print("Dependencies imported successfully")

Dependencies imported successfully


### Initialize Base Objects
The LLM, the prompt maker, text data sampler and result store are initialized. Relevant parameters:
* `MODEL_NAME`: the string path that defines which model to invoke via `litellm` library. The format is `{provider}/{providers_model_string}`, wherein provider list is given in an earlier cell.
* `API_KEY_ENV_VAR`: the environment variable name that contains the API key for the provider. You need to have acquired this key from the provider; **do not** store the key as a string in the notebook, that is playing with fire.
* `CORPUS_DIR`: relative path to the place containing template text
* `DATABASE_NAME`: name of the result store file for all intermediate outputs, which enables restart
* `OUTPUT_DIR`: relative path to the directory for the final output file
* `FILE_APPENDIX`: additional string appended to final output file name; note that if the result store contains multiple final outputs, these are already disambiguated by a numeric index

In [22]:
#MODEL_NAME = 'together_ai/moonshotai/Kimi-K2-Instruct-0905'
#MODEL_NAME = 'mistral/mistral-large-2411'
MODEL_NAME = 'anthropic/claude-sonnet-4-5-20250929'
API_KEY_ENV_VAR = 'ANTHROPIC_API_KEY'
#MODEL_TEMPERATURE = 0.7
#API_KEY_ENV_VAR = 'TOGETHER_AI_API_KEY'

# Initialize LLM
llm_config = LLMConfig(
    model=MODEL_NAME,
    api_key=os.environ.get(API_KEY_ENV_VAR),
    #temperature=MODEL_TEMPERATURE
)
llm = LLM(llm_config)

In [23]:
CORPUS_DIR = Path("data/russell")
DATABASE_NAME = "russell_author_modeling_sonnet.db"
OUTPUT_DIR = Path("outputs/author_modeling")
FILE_APPENDIX = 'sonnet'

prompt_maker = PromptMaker()
sampler = DataSampler(CORPUS_DIR)
store = ResultStore(DATABASE_NAME)

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

Only run the following in case you want a clean result store; partial deletes and clean up possible.

In [24]:
#store.reset('all')

### Configuration: Select Samples for Analysis

For Stages 1-3, we'll analyze 3-5 samples to extract stable cross-text patterns.

In [25]:
# Select samples for Stage 1-3 analysis
# These should be diverse, substantial passages (500-800 words recommended)
ANALYSIS_SAMPLES = [
    "sample_001",
    "sample_002",
    "sample_003",
    "sample_004",
    "sample_005",
    # Add more as needed for richer cross-text synthesis
]

# For this demo, we'll generate samples if they don't exist
# In production, you might use specific pre-selected samples
NUM_SAMPLES = 5
SAMPLE_PARAGRAPH_LENGTH = 10

print(f"Will analyze {NUM_SAMPLES} samples through Stages 1-3")

Will analyze 5 samples through Stages 1-3


## Stage 1: Analytical Mining

Run three specialized analyses on each sample:
- **Implied Author**: What sensibility emerges from the prose?
- **Decision Patterns**: What compositional choices are made at key junctures?
- **Functional Texture**: How do surface features serve purpose?

### Generate or Load Samples

In [26]:
# Generate samples if they don't exist
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    
    # Skip if already saved
    if store.get_sample(sample_id):
        print(f"✓ {sample_id} already gathered")
        continue
        
    # Generate new sample
    segment = sampler.sample_segment(p_length=SAMPLE_PARAGRAPH_LENGTH)
    store.save_segment(sample_id, segment)
    print(f"Generated {sample_id}")

# Display first sample for reference
sample = store.get_sample("sample_001")
print(f"\nSample 001 preview:\n{sample['text']}...")

Generated sample_001
Generated sample_002
Generated sample_003
Generated sample_004
Generated sample_005

Sample 001 preview:
Apart from the special doctrines advocated by Kant, it is very common
among philosophers to regard what is _a priori_ as in some sense mental,
as concerned rather with the way we must think than with any fact of
the outer world. We noted in the preceding chapter the three principles
commonly called 'laws of thought'. The view which led to their being so
named is a natural one, but there are strong reasons for thinking
that it is erroneous. Let us take as an illustration the law of
contradiction. This is commonly stated in the form 'Nothing can both be
and not be', which is intended to express the fact that nothing can at
once have and not have a given quality. Thus, for example, if a tree
is a beech it cannot also be not a beech; if my table is rectangular it
cannot also be not rectangular, and so on.

Now what makes it natural to call this principle a law of _t

### Run Stage 1A: Implied Author Analysis

In [27]:
# Analyze each sample for implied author
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    analyst_name = ImpliedAuthorConfig.analyst_name()
    
    # Check if analysis already exists (resume support)
    if store.get_analysis(sample_id, analyst_name):
        print(f"{sample_id}: {analyst_name} analysis already complete")
        continue
    
    # Get sample text
    sample = store.get_sample(sample_id)
    
    # Generate prompt
    config = ImpliedAuthorConfig(text=sample['text'])
    prompt = prompt_maker.render(config)
    
    # Run LLM
    print(f"{sample_id}: Running {analyst_name} analysis...")
    response = llm.complete(prompt)
    
    # Save analysis
    store.save_analysis(
        sample_id=sample_id,
        analyst=analyst_name,
        output=response.content,
        model=response.model
    )
    print(f"{sample_id}: {analyst_name} analysis complete")

print("\nStage 1A complete: All samples analyzed for implied author")

sample_001: Running implied_author analysis...
sample_001: implied_author analysis complete
sample_002: Running implied_author analysis...
sample_002: implied_author analysis complete
sample_003: Running implied_author analysis...
sample_003: implied_author analysis complete
sample_004: Running implied_author analysis...
sample_004: implied_author analysis complete
sample_005: Running implied_author analysis...
sample_005: implied_author analysis complete

Stage 1A complete: All samples analyzed for implied author


### Run Stage 1B: Decision Pattern Analysis

In [28]:
# Analyze each sample for decision patterns
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    analyst_name = DecisionPatternConfig.analyst_name()
    
    if store.get_analysis(sample_id, analyst_name):
        print(f"{sample_id}: {analyst_name} analysis already complete")
        continue
    
    sample = store.get_sample(sample_id)
    config = DecisionPatternConfig(text=sample['text'])
    prompt = prompt_maker.render(config)
    
    print(f"{sample_id}: Running {analyst_name} analysis...")
    response = llm.complete(prompt)
    
    store.save_analysis(
        sample_id=sample_id,
        analyst=analyst_name,
        output=response.content,
        model=response.model
    )
    print(f"{sample_id}: {analyst_name} analysis complete")

print("\nStage 1B complete: All samples analyzed for decision patterns")

sample_001: Running decision_pattern analysis...
sample_001: decision_pattern analysis complete
sample_002: Running decision_pattern analysis...
sample_002: decision_pattern analysis complete
sample_003: Running decision_pattern analysis...
sample_003: decision_pattern analysis complete
sample_004: Running decision_pattern analysis...
sample_004: decision_pattern analysis complete
sample_005: Running decision_pattern analysis...
sample_005: decision_pattern analysis complete

Stage 1B complete: All samples analyzed for decision patterns


### Run Stage 1C: Functional Texture Analysis

In [29]:
# Analyze each sample for functional texture
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    analyst_name = FunctionalTextureConfig.analyst_name()
    
    if store.get_analysis(sample_id, analyst_name):
        print(f"{sample_id}: {analyst_name} analysis already complete")
        continue
    
    sample = store.get_sample(sample_id)
    config = FunctionalTextureConfig(text=sample['text'])
    prompt = prompt_maker.render(config)
    
    print(f"{sample_id}: Running {analyst_name} analysis...")
    response = llm.complete(prompt)
    
    store.save_analysis(
        sample_id=sample_id,
        analyst=analyst_name,
        output=response.content,
        model=response.model
    )
    print(f"{sample_id}: {analyst_name} analysis complete")

print("\nStage 1C complete: All samples analyzed for functional texture")

sample_001: Running functional_texture analysis...
sample_001: functional_texture analysis complete
sample_002: Running functional_texture analysis...
sample_002: functional_texture analysis complete
sample_003: Running functional_texture analysis...
sample_003: functional_texture analysis complete
sample_004: Running functional_texture analysis...
sample_004: functional_texture analysis complete
sample_005: Running functional_texture analysis...
sample_005: functional_texture analysis complete

Stage 1C complete: All samples analyzed for functional texture


### Inspect Stage 1 Results

In [30]:
# Display completion status
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    sample, analyses = store.get_sample_with_analyses(sample_id)
    print(f"{sample_id}: {len(analyses)} analyses complete")
    for analyst_name in analyses.keys():
        print(f"  - {analyst_name}")

# Optionally display one analysis
print("\n--- Sample Implied Author Analysis (first 500 chars) ---")
sample, analyses = store.get_sample_with_analyses("sample_005")
implied_author = analyses.get(ImpliedAuthorConfig.analyst_name(), "Not found")
print(f"Total chars: {len(implied_author)}")
print(implied_author)

sample_001: 3 analyses complete
  - decision_pattern
  - functional_texture
  - implied_author
sample_002: 3 analyses complete
  - decision_pattern
  - functional_texture
  - implied_author
sample_003: 3 analyses complete
  - decision_pattern
  - functional_texture
  - implied_author
sample_004: 3 analyses complete
  - decision_pattern
  - functional_texture
  - implied_author
sample_005: 3 analyses complete
  - decision_pattern
  - functional_texture
  - implied_author

--- Sample Implied Author Analysis (first 500 chars) ---
Total chars: 16382
# IMPLIED AUTHOR CONSTRUCTION

## PART 1: DIMENSIONAL ANALYSIS

### 1. RELATIONSHIP TO MATERIAL

**Observation**: This author approaches philosophical problems with the stance of a diagnostician identifying disease—specifically, the disease of inherited error. The writing exhibits intellectual confidence combined with reformist zeal; the author is not discovering so much as *correcting*, dismantling structures that persist through inertia rathe

## Stage 2: Cross-Text Synthesis

Synthesize each analytical dimension across samples to identify stable patterns.

### Stage 2A: Implied Author Synthesis

In [31]:
# Collect all implied author analyses
implied_author_analyses = {}
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    analysis = store.get_analysis(sample_id, ImpliedAuthorConfig.analyst_name())
    if analysis:
        implied_author_analyses[sample_id] = analysis

# Check if synthesis already exists
synthesis_type = ImpliedAuthorSynthesisConfig.synthesis_type()
existing = store.list_syntheses(synthesis_type)

if existing:
    print(f"Implied author synthesis already exists: {existing[0]}")
    implied_author_synthesis_id = existing[0]
else:
    # Generate synthesis prompt
    config = ImpliedAuthorSynthesisConfig(
        implied_author_analyses=implied_author_analyses
    )
    prompt = prompt_maker.render(config)
    
    # Run synthesis
    print("Running implied author synthesis...")
    response = llm.complete(prompt)
    
    # Save synthesis
    sample_contributions = [
        (sample_id, ImpliedAuthorConfig.analyst_name())
        for sample_id in implied_author_analyses.keys()
    ]
    
    implied_author_synthesis_id = store.save_synthesis(
        synthesis_type=synthesis_type,
        output=response.content,
        model=response.model,
        sample_contributions=sample_contributions,
        config=config
    )
    print(f"Synthesis saved: {implied_author_synthesis_id}")

# Display preview
synth = store.get_synthesis(implied_author_synthesis_id)
print(f"\nPreview (first 400 chars):\n{synth['output'][:400]}...")

Running implied author synthesis...
Synthesis saved: implied_author_synthesis_001

Preview (first 400 chars):
# SYNTHETIC IMPLIED AUTHOR PORTRAIT

## PART 1: THE STABLE CORE

### Fundamental Stance Toward Ideas and Inquiry

This author consistently engages with subject matter as **problems yielding to patient analysis rather than mysteries requiring special insight**. Whether addressing logical principles, civilizational patterns, or scientific methodology, the stance is that of someone who has achieved c...


### Stage 2B: Decision Pattern Synthesis

In [32]:
# Collect all decision pattern analyses
decision_pattern_analyses = {}
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    analysis = store.get_analysis(sample_id, DecisionPatternConfig.analyst_name())
    if analysis:
        decision_pattern_analyses[sample_id] = analysis

# Check if synthesis already exists
synthesis_type = DecisionPatternSynthesisConfig.synthesis_type()
existing = store.list_syntheses(synthesis_type)

if existing:
    print(f"Decision pattern synthesis already exists: {existing[0]}")
    decision_pattern_synthesis_id = existing[0]
else:
    config = DecisionPatternSynthesisConfig(
        decision_pattern_analyses=decision_pattern_analyses
    )
    prompt = prompt_maker.render(config)
    
    print("Running decision pattern synthesis...")
    response = llm.complete(prompt)
    
    sample_contributions = [
        (sample_id, DecisionPatternConfig.analyst_name())
        for sample_id in decision_pattern_analyses.keys()
    ]
    
    decision_pattern_synthesis_id = store.save_synthesis(
        synthesis_type=synthesis_type,
        output=response.content,
        model=response.model,
        sample_contributions=sample_contributions,
        config=config
    )
    print(f"Synthesis saved: {decision_pattern_synthesis_id}")

synth = store.get_synthesis(decision_pattern_synthesis_id)
print(f"\nPreview (first 400 chars):\n{synth['output'][:400]}...")

Running decision pattern synthesis...
Synthesis saved: decision_pattern_synthesis_001

Preview (first 400 chars):
# SYNTHESIS: COMPOSITIONAL DECISION PATTERNS

## PART 1: COMPOSITIONAL PROBLEM TAXONOMY

### 1. **Opening Complex Arguments**
Establishing the stakes, scope, and oppositional frame of a philosophical argument without losing readers in abstraction or appearing to strawman opposing views.

### 2. **Making Abstract Distinctions Concrete**
Rendering philosophical differences (thought vs. thing, episte...


### Stage 2C: Textural Synthesis

In [33]:
# Collect all functional texture analyses
textural_analyses = {}
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    analysis = store.get_analysis(sample_id, FunctionalTextureConfig.analyst_name())
    if analysis:
        textural_analyses[sample_id] = analysis

# Check if synthesis already exists
synthesis_type = TexturalSynthesisConfig.synthesis_type()
existing = store.list_syntheses(synthesis_type)

if existing:
    print(f"Textural synthesis already exists: {existing[0]}")
    textural_synthesis_id = existing[0]
else:
    config = TexturalSynthesisConfig(
        textural_analyses=textural_analyses
    )
    prompt = prompt_maker.render(config)
    
    print("Running textural synthesis...")
    response = llm.complete(prompt)
    
    sample_contributions = [
        (sample_id, FunctionalTextureConfig.analyst_name())
        for sample_id in textural_analyses.keys()
    ]
    
    textural_synthesis_id = store.save_synthesis(
        synthesis_type=synthesis_type,
        output=response.content,
        model=response.model,
        sample_contributions=sample_contributions,
        config=config
    )
    print(f"Synthesis saved: {textural_synthesis_id}")

synth = store.get_synthesis(textural_synthesis_id)
print(f"\nPreview (first 400 chars):\n{synth['output'][:400]}...")

Running textural synthesis...
Synthesis saved: textural_synthesis_001

Preview (first 400 chars):
# SYNTHESIS: CHARACTERISTIC PROSE TEXTURE

## PART 1: SENTENCE ARCHITECTURE

### Structural Character

This author builds sentences as **instruments of progressive clarification**. The characteristic movement is **assertion followed by elaborative refinement**—a main clause establishes a position or observation, then the sentence unfolds rightward through qualifying phrases, contrastive elements, ...


## Stage 3: Author Model Creation
At the final stage, the outputs from steps 2 are joined and integrated into a prescriptive text that can be used to imbue another LLM with the writing style and sensibilities of the author of the analyzed texts. This yields the final output, which in addition to being stored in the database, is exported as a meta data enhanced markdown file.

In [34]:
implied_synth = store.get_synthesis(implied_author_synthesis_id)
decision_synth = store.get_synthesis(decision_pattern_synthesis_id)
textural_synth = store.get_synthesis(textural_synthesis_id)

# Check if author model definition already exists
synthesis_type = AuthorModelDefinitionConfig.synthesis_type()
existing = store.list_syntheses(synthesis_type)

if existing:
    print(f"Author model definition already exists: {existing[0]}")
    author_model_id = existing[0]
else:
    # Generate author model definition
    config = AuthorModelDefinitionConfig(
        implied_author_synthesis=implied_synth['output'],
        decision_pattern_synthesis=decision_synth['output'],
        textural_synthesis=textural_synth['output']
    )
    prompt = prompt_maker.render(config)

    print("Constructing author model definition...")
    response = llm.complete(prompt)

    # Save with parent linkage (no direct sample contributions)
    author_model_id = store.save_synthesis(
        synthesis_type=synthesis_type,
        output=response.content,
        model=response.model,
        sample_contributions=[],  # Inherits from parents
        config=config,
        parent_synthesis_id=implied_author_synthesis_id  # Link to one parent
    )
    print(f"Author model definition saved: {author_model_id}")

# Display preview
author_model = store.get_synthesis(author_model_id)
print(f"\nTotal chars of Author Model: {len(author_model['output'])}")
print(f"\nAuthor Model Preview (first 800 chars):\n{author_model['output'][:800]}...")


author_model_path = OUTPUT_DIR / f"{author_model_id}_{FILE_APPENDIX}.txt"
store.export_synthesis(
    synthesis_id=author_model_id,
    output_path=author_model_path,
    metadata_format='yaml'
)
print(f"\nAuthor model exported to: {author_model_path}")

Constructing author model definition...
Author model definition saved: author_model_definition_001

Total chars of Author Model: 20176

Author Model Preview (first 800 chars):
# AUTHOR MIND MODEL: BERTRAND RUSSELL

---

## PART 1: THE GENERATIVE STANCE

When you write as this author, you inhabit a position of **clarity achieved and being transmitted**. You are not discovering in real-time; you have thought through the confusions, identified where thinking goes wrong, and now demonstrate the path to understanding. Your relationship to your material is that of someone who has **solved the puzzle and now shows others the solution**—not with condescension, but with the confidence that clear thinking is accessible to anyone willing to follow the reasoning.

**Your orientation to material** is fundamentally diagnostic. When you encounter a philosophical problem or cultural phenomenon, you ask: *Where does the confusion originate? What historical accident or logical er...

Author model export